In [1]:
import openai

In [ ]:
from openai import OpenAI
import os
#print(os.getenv("OPENAI_API_KEY"))
os.environ

In [4]:
client = OpenAI()

In [6]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "Is it too late to join the llm_zoomcamp course?" }]
)

In [8]:
response.choices[0].message.content

"As of my last update in October 2023, I don't have specific information about the current status or registration deadlines for the LLM (Large Language Model) Zoomcamp course. The timing and availability for joining such courses can vary depending on the organization offering them.\n\nTo find out if it's too late to join the LLM Zoomcamp course, you should:\n\n1. **Check the Official Website or Communication Channels**: Look for any announcements or updates regarding enrollment deadlines or late registration policies.\n\n2. **Contact the Organizers**: Reach out directly to the organizers or facilitators of the course. They can provide the most accurate and up-to-date information regarding availability and registration.\n\n3. **Look for Community Discussions**: If there's a community forum or social media group related to the course, it might have information from other participants or updates from the organizers.\n\nIf you provide more specific details about the course provider, I migh

In [1]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch(
    "http://localhost:9200",
    verify_certs=False
)

print(es_client.info())

{'name': '0b3cfbec6f5f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'efDPJR5yQIqHUDFVxMkfLQ', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [7]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines 

In [10]:
from tqdm.auto import tqdm

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name  = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/opt/homebrew/Caskroom/miniforge/base/envs/llm_zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/_r/_94p_t6s1933ynymfxl25_7h0000gn/T/ipykernel_13289/1308179079.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)
100%|██████████| 948/948 [00:07<00:00, 120.58it/s]


In [26]:
query = 'How do execute a command on a Kubernetes pod?'
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"], #Matches in the question field count 4× as much (score-wise) as matches in the text field.
                        "type": "best_fields"
                    }
                },
                #"filter": {
                #    "term": {
                #        "course": "data-engineering-zoomcamp"
                #    }
                #}
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)
result_docs = []
#print(response)
print('___________________')
print('Score: ',response['hits']['hits'][0]['_score'])
print('Answer: ', response['hits']['hits'][0]['_source'])
#for hit in response['hits']['hits']:
#    result_docs.append(hit['_source'])

___________________
Score:  44.50556
Answer:  {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}


/var/folders/_r/_94p_t6s1933ynymfxl25_7h0000gn/T/ipykernel_13289/2364632510.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [42]:
query_2 = 'How do copy a file to a Docker container?'
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)
result_docs = []
#print(response)
print('___________________')
for hit in response['hits']['hits']:
    source = hit['_source']
    question = source['question']
    text = source['text']
    result_docs.append([question, text])


print(*result_docs, sep="\n")

___________________
['How do I debug a docker container?', 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)']
['Kubernetes-dashboard', 'Deploy and Access the Kubernetes Dashboard\nLuke']
['How do I copy files from a different folder into docker container’s working directory?', 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan']


/var/folders/_r/_94p_t6s1933ynymfxl25_7h0000gn/T/ipykernel_13289/1071363777.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [57]:
context_template = """
Q: {question}
A: {text}
""".strip()


context_docs = []
for question, text in result_docs:

    doc = context_template.format(question=question, text=text).strip()
    context_docs.append(doc)

context = "\n\n".join(context_docs)
print(context)



Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: Kubernetes-dashboard
A: Deploy and Access the Kubernetes Dashboard
Luke

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:
COPY ["src/predict.py", "models/xgb_model.bin", "./"]											Gopakumar Gopinathan


In [58]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
question = 'How do I execute a command in a running docker container?'
prompt = (prompt_template.format(question=question, context=context).strip())
print(prompt)
print(len(prompt))


You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: Kubernetes-dashboard
A: Deploy and Access the Kubernetes Dashboard
Luke

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic synta

In [61]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
enc_prompt = encoding.encode(prompt)
print(len(enc_prompt))



242
